In [ ]:
import optuna

def objective(trial):
    # Hyperparameters to be tuned
    lr = trial.suggest_float('lr', 1e-5, 1e-3, log=True)
    embedding_dim = trial.suggest_categorical('embedding_dim', [16, 32, 64, 128])
    num_hidden_units = trial.suggest_categorical('num_hidden_units', [16, 32, 64, 128])
    model = GCN(num_nodes, embedding_dim, num_classes, num_hidden_units)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.CrossEntropyLoss()
    model = model.to(device)

    # Early stopping and scheduler setup
    patience = 5
    best_val_loss = float('inf')
    epochs_no_improve = 0
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)
    
    # Training loop
    for epoch in range(100):
        train_loss = train()
        val_loss = validate()

        scheduler.step(val_loss)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
            best_model_state = model.state_dict()  # Save the best model state for this trial
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= patience:
            break  # Early stopping

    # Save the best model state of this trial to a file
    trial_model_path = f"model_trial_{trial.number}.pth"
    torch.save(best_model_state, trial_model_path)

    # After training, clear memory
    del model, optimizer, criterion
    torch.cuda.empty_cache()

    # Return the best validation loss of this trial
    return best_val_loss


study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)  # Set the number of trials

print("Best hyperparameters: ", study.best_params)

In [ ]:
import optuna

def objective(trial):
    # Hyperparameters to be tuned with expanded ranges
    embedding_dim = trial.suggest_categorical('embedding_dim', [512, 1024])
    num_hidden_units = trial.suggest_categorical('num_hidden_units', [512, 1024])

    # Learning rate
    lr = trial.suggest_float('lr', 1e-5, 1e-3, log=True)

    # Model setup
    model = GCN(num_nodes, embedding_dim, num_classes, num_hidden_units)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.CrossEntropyLoss()
    model = model.to(device)

    # Early stopping and scheduler setup
    patience = 5
    best_val_loss = float('inf')
    epochs_no_improve = 0
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)
    
    # Training loop
    for epoch in range(100):
        train_loss = train()
        val_loss = validate()

        scheduler.step(val_loss)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
            best_model_state = model.state_dict()  # Save the best model state for this trial
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= patience:
            break  # Early stopping

    # Save the best model state of this trial to a file
    trial_model_path = f"model_trial_{trial.number}_expanded_range.pth"
    torch.save(best_model_state, trial_model_path)

    # Return the best validation loss of this trial
    return best_val_loss

# Create and run the study
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=5)  # Set the number of trials

print("Best hyperparameters: ", study.best_params)
